In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('study_dataset.csv', encoding='utf-8', sep='\t', index_col=0)
data.head()

,business_id,name,stars,text
882258,TyYpzJzeKWtyMdPFtPRUEg,Prince Pizza,1,This place was horrible! They got my entire or...
642407,LCMpkeQWHdCbuX0Nlkmeow,Jack in the Box,1,I have no idea why I bother. It's the closest ...
831418,AZ42H2kk4JUiskC9NPoUjw,Hobnobber's Variety Bar & Restaurant,1,After reading glowing reviews of staff and foo...
452134,0CeoONLXqseUxPdGerefOQ,Mexican Post,1,Horrible Ground beef/bean Chimichanga. There w...
921422,QYEiZzwL8I4-q_3U0bq0eQ,The Nutrition Factory,1,They are usually awesome but picked up a kale ...


In [4]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from sklearn.model_selection import ParameterGrid
from gensim.models.coherencemodel import CoherenceModel

In [5]:
# Préparation des données
documents = ['Premier document', 'Deuxième document', 'Troisième document', ...]  # Vos documents
corpus = [ "service  lent  inefficace",
    " nourriture  froide  fade",
    "Les prix   élevés   qualité offerte",
    " Attendre longtemps être servi.",
    " personnel  impoli  désagréable. Attente  importante",
    " Chocolat pas terrible. Chocolat Aucun Goût",
    " Chocolat perimé "
]
documents = [d.split() for d in corpus]

In [11]:
documents

[['service', 'lent', 'inefficace'],
 ['nourriture', 'froide', 'fade'],
 ['Les', 'prix', 'élevés', 'qualité', 'offerte'],
 ['Attendre', 'longtemps', 'être', 'servi.'],
 ['personnel', 'impoli', 'désagréable.', 'Attente', 'importante'],
 ['Chocolat', 'pas', 'terrible.', 'Chocolat', 'Aucun', 'Goût'],
 ['Chocolat', 'perimé']]

In [6]:
# Création du dictionnaire
dictionary = Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [12]:
dictionary

In [10]:
corpus

[[(0, 1), (1, 1), (2, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1)],
 [(11, 1), (12, 1), (13, 1), (14, 1)],
 [(15, 1), (16, 1), (17, 1), (18, 1), (19, 1)],
 [(20, 1), (21, 2), (22, 1), (23, 1), (24, 1)],
 [(21, 1), (25, 1)]]

In [7]:
# Liste des nombres de sujets à tester
num_topics_list = [5, 10, 15, 20]

# Définition des paramètres à tester
param_grid = {
    'num_topics': [5, 10, 15],
    'passes': [10, 20, 30]
}

In [8]:
# Liste pour stocker les résultats
results = []
# Variables pour stocker le meilleur modèle et son score de cohérence
best_model = None
best_coherence_score = -1

# Grid Search
for params in ParameterGrid(param_grid):
    # Entraînement du modèle LDA avec les paramètres donnés
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, **params)
    
    # Calcul de la cohérence des sujets
    coherence_model = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    
    # Stockage des résultats
    results.append((params, coherence_score))
    
    # Mise à jour du meilleur modèle si le score de cohérence est plus élevé
    if coherence_score > best_coherence_score:
        best_model = lda_model
        best_coherence_score = coherence_score    

In [9]:
# Affichage du meilleur modèle et de son score de cohérence
print("Meilleur modèle :")
print(best_model)
print("Score de cohérence des sujets :", best_coherence_score)

print("\n", "=====================================", "\n")

# Affichage des thèmes pour best_model
topics = best_model.print_topics(num_topics=5, num_words=10)
for topic in topics:
    print(topic)

Meilleur modèle :
LdaModel<num_terms=26, num_topics=10, decay=0.5, chunksize=2000>
Score de cohérence des sujets : 0.6299162827260991


(0, '0.145*"impoli" + 0.145*"personnel" + 0.145*"importante" + 0.145*"Attente" + 0.145*"désagréable." + 0.013*"Chocolat" + 0.013*"service" + 0.013*"nourriture" + 0.013*"perimé" + 0.013*"fade"')
(9, '0.196*"lent" + 0.196*"inefficace" + 0.196*"service" + 0.018*"Chocolat" + 0.018*"perimé" + 0.018*"nourriture" + 0.018*"fade" + 0.018*"froide" + 0.018*"Attendre" + 0.018*"être"')
(4, '0.167*"longtemps" + 0.167*"être" + 0.167*"servi." + 0.167*"Attendre" + 0.015*"Chocolat" + 0.015*"service" + 0.015*"fade" + 0.015*"nourriture" + 0.015*"perimé" + 0.015*"inefficace"')
(8, '0.244*"Chocolat" + 0.128*"Goût" + 0.128*"Aucun" + 0.128*"terrible." + 0.128*"pas" + 0.012*"service" + 0.012*"lent" + 0.012*"Attendre" + 0.012*"fade" + 0.012*"perimé"')
(7, '0.038*"Chocolat" + 0.038*"service" + 0.038*"perimé" + 0.038*"fade" + 0.038*"froide" + 0.038*"nourriture" + 0.038*"lent" + 0